In [3]:
"""
进行二分类
"""

'\n进行二分类\n'

In [4]:
import torch
import torch.nn as nn 
import torch.optim 
import pandas as pd 
import numpy as np

In [5]:
np.random.seed(0)
X_train_fpath = './data/X_train'
Y_train_fpath = './data/Y_train'
X_test_fpath = './data/X_test'
output_fpath = './output_{}.csv'

# Parse csv files to numpy array
with open(X_train_fpath) as f:
    next(f)
    X_train = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)
with open(Y_train_fpath) as f:
    next(f)
    Y_train = np.array([line.strip('\n').split(',')[1] for line in f], dtype = float)
with open(X_test_fpath) as f:
    next(f)
    X_test = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)

In [6]:
X_train.shape

(54256, 510)

In [7]:
X_test.shape

(27622, 510)

In [8]:
Y_train.shape

(54256,)

In [9]:
import numpy as np

np.random.seed(0)
X_train_fpath = './data/X_train'
Y_train_fpath = './data/Y_train'
X_test_fpath = './data/X_test'
output_fpath = './output_{}.csv'

# Parse csv files to numpy array
with open(X_train_fpath) as f:
    next(f)
    X_train = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)
with open(Y_train_fpath) as f:
    next(f)
    Y_train = np.array([line.strip('\n').split(',')[1] for line in f], dtype = float)
with open(X_test_fpath) as f:
    next(f)
    X_test = np.array([line.strip('\n').split(',')[1:] for line in f], dtype = float)

def _normalize(X, train = True, specified_column = None, X_mean = None, X_std = None):
    # This function normalizes specific columns of X.
    # The mean and standard variance of training data will be reused when processing testing data.
    #
    # Arguments:
    #     X: data to be processed
    #     train: 'True' when processing training data, 'False' for testing data
    #     specific_column: indexes of the columns that will be normalized. If 'None', all columns
    #         will be normalized.
    #     X_mean: mean value of training data, used when train = 'False'
    #     X_std: standard deviation of training data, used when train = 'False'
    # Outputs:
    #     X: normalized data
    #     X_mean: computed mean value of training data
    #     X_std: computed standard deviation of training data

    if specified_column == None:
        specified_column = np.arange(X.shape[1])
    if train:
        X_mean = np.mean(X[:, specified_column] ,0).reshape(1, -1)
        X_std  = np.std(X[:, specified_column], 0).reshape(1, -1)

    X[:,specified_column] = (X[:, specified_column] - X_mean) / (X_std + 1e-8)
     
    return X, X_mean, X_std

def _train_dev_split(X, Y, dev_ratio = 0.25):
    # This function spilts data into training set and development set.
    train_size = int(len(X) * (1 - dev_ratio))
    return X[:train_size], Y[:train_size], X[train_size:], Y[train_size:]

# Normalize training and testing data
X_train, X_mean, X_std = _normalize(X_train, train = True)
X_test, _, _= _normalize(X_test, train = False, specified_column = None, X_mean = X_mean, X_std = X_std)
    
# Split data into training set and development set
dev_ratio = 0.1
X_train, Y_train, X_dev, Y_dev = _train_dev_split(X_train, Y_train, dev_ratio = dev_ratio)

train_size = X_train.shape[0]
dev_size = X_dev.shape[0]
test_size = X_test.shape[0]
data_dim = X_train.shape[1]
print('Size of training set: {}'.format(train_size))
print('Size of development set: {}'.format(dev_size))
print('Size of testing set: {}'.format(test_size))
print('Dimension of data: {}'.format(data_dim))

Size of training set: 48830
Size of development set: 5426
Size of testing set: 27622
Dimension of data: 510


In [22]:
#model
class Model(nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.fc = nn.Linear(510,2)
        self.fc2 = nn.Linear(100,2)
        self.sig = nn.ReLU()
    
    def forward(self,x):
        out = self.fc(x)
        out = self.sig(out)
        # out = self.fc2(out)
        # out = self.sig(out)
        return out
    
model =Model()
optimizer= torch.optim.Adam(model.parameters(),lr =0.05)
cirection = nn.CrossEntropyLoss()

In [23]:
epochs= 100
#尝试小批量？
#48330 -1000
# X_train, Y_train = _shuffle(X_train, Y_train)
# X_train = torch.from_numpy(X_train).float()
# Y_train = torch.from_numpy(Y_train).float()
# X_dev = torch.from_numpy(X_dev).float()
# Y_dev = torch.from_numpy(Y_dev).float()
# train_size =48330
# batch_size =256
for epoch in range(epochs):
    # for idx in range(int(np.floor(train_size / batch_size))):
    #     X = X_train[idx*batch_size:(idx+1)*batch_size]
    #     Y = Y_train[idx*batch_size:(idx+1)*batch_size]
        # print(type(X))
    y_lr_pred = model(X_train)
    loss = cirection(y_lr_pred,Y_train.long())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        #acc = _accuracy(y_lr_pred.detach.numpy(),Y_train.detach.numpy())
    
    model.eval()
    with torch.no_grad():
        y_dlr_pred = model(X_dev)
        testloss = cirection(y_dlr_pred,Y_dev.long())
    if(epoch+1)%10==0:
        print("loss: {:2f} test loss: {:2f}".format(loss.item(),testloss.item()))

loss: 0.386796 test loss: 0.426480
loss: 0.331405 test loss: 0.378263
loss: 0.304998 test loss: 0.363038
loss: 0.284545 test loss: 0.343114
loss: 0.272068 test loss: 0.389100
loss: 0.262830 test loss: 0.367377
loss: 0.246557 test loss: 0.421542
loss: 0.240706 test loss: 0.448523
loss: 0.231967 test loss: 0.456239
loss: 0.230978 test loss: 0.510206


In [50]:
def _shuffle(X,Y):
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
    return (X[randomize],Y[randomize])

def _sigmoid(z):
    return np.clip(1/(1.0+np.exp(-z)),1e-8,1-(1e-8))

def _f(X,w,b):
    return _sigmoid(np.matmul(X,w)+b)

def _predict(X,w,b):
    return np.round(_f(X,w,b)).astype(np.int)

def _accuracy(Y_pred,Y_label):
    acc = 1-np.mean(np.abs(Y_pred-Y_label))
    return acc


In [51]:
def _cross_entropy_loss(y_pred,Y_label):
    cross_entropy = -np.dot(Y_label,np.log(y_pred))-np.dot((1-Y_label),np.log(1-y_pred))
    return cross_entropy
def _gradient(X,Y_label,w,b):
    y_pred = _f(X,w,b)
    pred_error = Y_label-y_pred
    w_grad = -np.sum(pred_error*X.T,1)
    b_grad  = -np.sum(pred_error)
    return w_grad,b_grad

In [67]:
# Zero initialization for weights ans bias
w = np.zeros((data_dim,)) 
b = np.zeros((1,))

# Some parameters for training    
max_iter = 10
batch_size = 8
learning_rate = 0.2

# Keep the loss and accuracy at every iteration for plotting
train_loss = []
dev_loss = []
train_acc = []
dev_acc = []

# Calcuate the number of parameter updates
step = 1
X_train, Y_train = X_train.numpy(),Y_train.numpy()
# Iterative training
for epoch in range(max_iter):
    # Random shuffle at the begging of each epoch
    X_train, Y_train = _shuffle(X_train, Y_train)
        
    # Mini-batch training
    for idx in range(int(np.floor(train_size / batch_size))):
        X = X_train[idx*batch_size:(idx+1)*batch_size]
        Y = Y_train[idx*batch_size:(idx+1)*batch_size]

        # Compute the gradient
        w_grad, b_grad = _gradient(X, Y, w, b)
            
        # gradient descent update
        # learning rate decay with time
        w = w - learning_rate/np.sqrt(step) * w_grad
        b = b - learning_rate/np.sqrt(step) * b_grad

        step = step + 1
            
    # Compute loss and accuracy of training set and development set
    y_train_pred = _f(X_train, w, b)
    Y_train_pred = np.round(y_train_pred)
    train_acc.append(_accuracy(Y_train_pred, Y_train))
    train_loss.append(_cross_entropy_loss(y_train_pred, Y_train) / train_size)

    y_dev_pred = _f(X_dev, w, b)
    Y_dev_pred = np.round(y_dev_pred)
    dev_acc.append(_accuracy(Y_dev_pred, Y_dev))
    dev_loss.append(_cross_entropy_loss(y_dev_pred, Y_dev) / dev_size)

print('Training loss: {}'.format(train_loss[-1]))
print('Development loss: {}'.format(dev_loss[-1]))
print('Training accuracy: {}'.format(train_acc[-1]))
print('Development accuracy: {}'.format(dev_acc[-1]))

Training loss: 0.27051389197026615
Development loss: 0.2981100244597504
Training accuracy: 0.8847020274421462
Development accuracy: 0.8772576483597494


In [ ]:
import matplotlib.pyplot as plt

# Loss curve
plt.plot(train_loss)
plt.plot(dev_loss)
plt.title('Loss')
plt.legend(['train', 'dev'])
plt.savefig('loss.png')
plt.show()

# Accuracy curve
plt.plot(train_acc)
plt.plot(dev_acc)
plt.title('Accuracy')
plt.legend(['train', 'dev'])
plt.savefig('acc.png')
plt.show()